In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import math
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel

logging.getLogger(__name__)

<Logger __main__ (INFO)>

In [ ]:
t = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from utils.MIND import MIND

In [ ]:
config.reducer = 'bm25'
# config.reducer = 'match'

train_path = config.path + 'MIND/MINDdemo_dev/'
a = MIND(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [ ]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False)
records1 = list(loader1)

In [ ]:
records1[0]

In [ ]:
mind_path = 